### Pydantic 

Pydantic Models provides the richest feature set with field validation, description, and nested structures.

In [23]:
import os
from langchain.chat_models import init_chat_model

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model = init_chat_model(
  model="gpt-4o-mini",
)
model

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x000002716A7B2060>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002716A9A65D0>, root_client=<openai.OpenAI object at 0x000002716B306A20>, root_async_client=<openai.AsyncOpenAI object at 0x000002716AB52690>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

### Structured Format Schema:

In [24]:
from pydantic import BaseModel, Field


class Movie(BaseModel):
  title: str = Field(description="The title of the movie")
  year: int = Field(description="The year the movie was released")
  rating: float = Field(description="The rating of the movie")
  cast: list[str] = Field(description="The cast of the movie")
  
  


In [25]:
structured_model =model.with_structured_output(Movie)
structured_model

RunnableBinding(bound=ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x000002716A7B2060>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002716A9A65D0>, root_client=<openai.OpenAI object at 0x000002716B306A20>, root_async_client=<openai.AsyncOpenAI object at 0x000002716AB52690>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True), kwargs={'response_format': <class '__main__.Movie'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict'

In [26]:
# Without structured output
model.invoke("Provide the details about the movie Inception")

AIMessage(content='"Inception" is a science fiction film released in 2010, written and directed by Christopher Nolan. The film is known for its innovative storytelling, complex themes, and stunning visual effects.\n\n### Plot Summary\nThe story follows Dom Cobb (played by Leonardo DiCaprio), a skilled thief who specializes in the art of "extraction," a process of stealing valuable secrets from deep within the subconscious during the dream state. Cobb is offered a chance to have his criminal history erased as payment for a task considered to be impossible: "inception," which involves planting an idea into someone’s mind without their knowledge.\n\nCobb assembles a team that includes Arthur (Joseph Gordon-Levitt), his right-hand man; Ariadne (Elliot Page), a young architect who designs dreamscapes; Eames (Tom Hardy), a forger who can manipulate his appearance in dreams; and Yusuf (Dileep Rao), a chemist who helps them concoct a potent sedative. Their target is Robert Fischer (Cillian Mur

In [27]:
# With structured output schema
structured_model.invoke("Provide the details about the movie Inception")

Movie(title='Inception', year=2010, rating=8.8, cast=['Leonardo DiCaprio', 'Joseph Gordon-Levitt', 'Elliot Page', 'Tom Hardy', 'Ken Watanabe', 'Marion Cotillard', 'Cillian Murphy', 'Dileep Rao', 'Michael Caine'])

### Message output alongside Parsed Structure

In [28]:
structured_model =model.with_structured_output(Movie, include_raw=True)

response = structured_model.invoke("Provide the details about the movie Inception")
response




{'raw': AIMessage(content='{"title":"Inception","year":2010,"rating":8.8,"cast":["Leonardo DiCaprio","Joseph Gordon-Levitt","Elliot Page","Tom Hardy","Ken Watanabe","Dileep Rao","Cillian Murphy","Marion Cotillard","Michael Caine"]}', additional_kwargs={'parsed': Movie(title='Inception', year=2010, rating=8.8, cast=['Leonardo DiCaprio', 'Joseph Gordon-Levitt', 'Elliot Page', 'Tom Hardy', 'Ken Watanabe', 'Dileep Rao', 'Cillian Murphy', 'Marion Cotillard', 'Michael Caine']), 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 124, 'total_tokens': 187, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3683ee3deb', 'id': 'chatcmpl-CxuhyKMPiUFz18bxI1WeWem8FM3YU', 'service_tier': 'default', 'finish_reason

### Nested Structure

In [29]:
from pydantic import BaseModel, Field

class Actor(BaseModel):
  name: str = Field(description="The name of the actor")
  role: str = Field(description="The role of the actor in the movie")
  age: int = Field(description="The age of the actor")

class Movie(BaseModel):
  title: str = Field(description="The title of the movie")
  year: int = Field(description="The year the movie was released")
  # Nested structure
  cast: list[Actor] = Field(description="The cast of the movie")


  

In [30]:
model_with_structure = model.with_structured_output(Movie)

response = model_with_structure.invoke("Provide the details about the movie Inception")
response

Movie(title='Inception', year=2010, cast=[Actor(name='Leonardo DiCaprio', role='Dom Cobb', age=45), Actor(name='Joseph Gordon-Levitt', role='Arthur', age=39), Actor(name='Elliot Page', role='Ariadne', age=36), Actor(name='Tom Hardy', role='Eames', age=43), Actor(name='Ken Watanabe', role='Saito', age=63), Actor(name='Cillian Murphy', role='Robert Fischer', age=47), Actor(name='Marion Cotillard', role='Mal Cobb', age=48), Actor(name='Michael Caine', role='Professor Stephen Miles', age=90)])

### TypedDict

TypedDict provides a simpler alternative using Python's built-in typing, ideal when you don't need runtime validation

In [44]:
from typing import TypedDict, Annotated

class MovieDict(TypedDict):
  """A movie with details"""
  title: Annotated[str, ..., "The title of the movie"]
  year: Annotated[ int, ...,   "The year the movie was released"]
  rating: Annotated[ float, ..., "The rating of the movie"]
 


In [46]:
model_with_typed_dict = model.with_structured_output(MovieDict)

response = model_with_typed_dict.invoke("Provide the details about the movie Avengers: Endgame")
response


{'title': 'Avengers: Endgame', 'year': 2019, 'rating': 8.4}

In [47]:
class Actor(TypedDict):
  name: str = Field(description="The name of the actor")
  role: str = Field(description="The role of the actor in the movie")
  age: int = Field(description="The age of the actor")

class Movie(TypedDict):
  title: str = Field(description="The title of the movie")
  year: int = Field(description="The year the movie was released")
  # Nested structure
  cast: list[Actor] = Field(description="The cast of the movie")

In [48]:
model_with_typed_dict = model.with_structured_output(Movie)

response = model_with_typed_dict.invoke("Provide the details about the movie Avengers: Endgame")
response

{'title': 'Avengers: Endgame',
 'year': 2019,
 'cast': [{'name': 'Robert Downey Jr.',
   'role': 'Tony Stark / Iron Man',
   'age': 54},
  {'name': 'Chris Evans', 'role': 'Steve Rogers / Captain America', 'age': 38},
  {'name': 'Scarlett Johansson',
   'role': 'Natasha Romanoff / Black Widow',
   'age': 34},
  {'name': 'Mark Ruffalo', 'role': 'Bruce Banner / Hulk', 'age': 52},
  {'name': 'Chris Hemsworth', 'role': 'Thor', 'age': 36},
  {'name': 'Jeremy Renner', 'role': 'Clint Barton / Hawkeye', 'age': 48},
  {'name': 'Don Cheadle', 'role': 'James Rhodes / War Machine', 'age': 55},
  {'name': 'Paul Rudd', 'role': 'Scott Lang / Ant-Man', 'age': 51},
  {'name': 'Brie Larson', 'role': 'Carol Danvers / Captain Marvel', 'age': 29},
  {'name': 'Karen Gillan', 'role': 'Nebula', 'age': 35},
  {'name': 'Benedict Wong', 'role': 'Wong', 'age': 52},
  {'name': 'Danai Gurira', 'role': 'Okoye', 'age': 44},
  {'name': 'Tom Holland', 'role': 'Peter Parker / Spider-Man', 'age': 27},
  {'name': 'Benedict

In [49]:
model.profile

{'max_input_tokens': 128000,
 'max_output_tokens': 16384,
 'image_inputs': True,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': False,
 'tool_calling': True,
 'structured_output': True,
 'image_url_inputs': True,
 'pdf_inputs': True,
 'pdf_tool_message': True,
 'image_tool_message': True,
 'tool_choice': True}

### Model with Pydantic

In [70]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
import os

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


class ContactInfo(BaseModel):
  """Contact informaion for a person"""
  name: str = Field(description="The name of the person")
  email: str = Field(description="The email address of the person")
  phone: str = Field(description="The phone number of the person")


model = init_chat_model("gpt-4o-mini")
# model = init_chat_model("groq:qwen/qwen3-32b")


agent = create_agent(
  model= model,
  response_format= ContactInfo,
  
  
)

user_input = "Extract the contact information from the following text: John Doe, john.doe@example.com, 123-456-7890"

result = agent.invoke({"messages": [{"role": "user", "content": user_input}]})


# result["messages"][-1].content

result["structured_response"]

# print(type(result["messages"][-1].content))

ContactInfo(name='John Doe', email='john.doe@example.com', phone='123-456-7890')

### Model with TypedDict

In [72]:
from typing import TypedDict, Annotated
from langchain.agents import create_agent

class ContactInfoDict(TypedDict):
  name: Annotated[str, ..., "The name of the person"]
  email: Annotated[str, ..., "The email address of the person"]
  phone: Annotated[str, ..., "The phone number of the person"]
  

model = init_chat_model("gpt-4o-mini")
# model = init_chat_model("groq:qwen/qwen3-32b")


agent = create_agent(
  model= model,
  response_format= ContactInfoDict,
  
  
)

user_input = "Extract the contact information from the following text: John Doe, john.doe@example.com, 123-456-7890"

result = agent.invoke({"messages": [{"role": "user", "content": user_input}]})


# result["messages"][-1].content

result["structured_response"]

print(type(result["messages"][-1].content))




<class 'str'>
